<a href="https://colab.research.google.com/github/edinaldoab/challenge_data_science_2/blob/main/notebooks/semanas_03_e_04_recomendador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Semanas 03 e 04

Durante as semanas 3 e 4 vamos focar em melhorar o sistema de recomendação da InsightPlaces.
Nosso sistema não está recebendo os clicks que esperava apenas recomendando imoveis das mesmas regiões e na mesma faixa de preço, por isso como parte do time de Data Science precisamos criar uma nova mecânica de recomendação de imóveis.

O objetivo é criar um sistema de recomendação de imóveis baseado em similaridade de características. A base de dados utilizada será a base de dados tratada na semana 2, após transformar as variáveis categóricas em variáveis binárias e antes de aplicarmos a vetorização.

# 1. Preparação do Ambiente

## a. Instalação das dependências

In [58]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [59]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

## b. Inicialização da SparkSession

In [60]:
import findspark
findspark.init()

In [61]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("recomendador") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

## c. Montagem do Drive

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## d. Inicializando a  UI do Spark 

In [63]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [64]:
with open('/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dados/authTokenngrok.txt', 'r') as file:
  content = file.read()
  file.close()

my_authtoken = str(content)

In [65]:
get_ipython().system_raw('./ngrok authtoken ' + my_authtoken)
get_ipython().system_raw('./ngrok http 4050 &')

!curl -s http://localhost:4040/api/tunnels

{"tunnels":[],"uri":"/api/tunnels"}


In [66]:
spark

Carregamento dos dados:

# 2. Carregamento e preparação dos dados

## a. Leitura do dataset

In [67]:
path='/content/drive/MyDrive/Colab Notebooks/projeto_data_science_imobni/dataset_ml_regressao'

data = spark.read.parquet(path)

In [68]:
data.show(5)

+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|         bairro|condominio|   iptu|    valor|zona_central|zona_norte|zona_oeste|zona_sul|Academia|animais_permitidos|Churrasqueira|condominio_fechado|Elevador|Piscina|Playground|portaria_24h|portao_eletronico|salao_de_festas|
+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00790b85-56e3-43a...|   11|    166.0|        5|      4|     4|   2|    Jacarepaguá|    2100.0| 4600.0|1750000.0|     

In [69]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: double (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- zona_central: integer (nullable = true)
 |-- zona_norte: integer (nullable = true)
 |-- zona_oeste: integer (nullable = true)
 |-- zona_sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- animais_permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- condominio_fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- portaria_24h: integer (nullable = true)
 |-- portao_eletronico: integer (nullable

In [70]:
print(f'Quantidade de linhas: {data.count()}')
print(f'Quantidade de colunas: {len(data.columns)}')

Quantidade de linhas: 66551
Quantidade de colunas: 25


Checagem de nulos: 

In [71]:
from pyspark.sql import functions as f

In [72]:
data.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in data.columns]).show()

+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
| id|andar|area_util|banheiros|quartos|suites|vaga|bairro|condominio|iptu|valor|zona_central|zona_norte|zona_oeste|zona_sul|Academia|animais_permitidos|Churrasqueira|condominio_fechado|Elevador|Piscina|Playground|portaria_24h|portao_eletronico|salao_de_festas|
+---+-----+---------+---------+-------+------+----+------+----------+----+-----+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|  0|    0|        0|        0|      0|     0|   0|     0|         0|   0|    0|           0|         0|         0|       0|       0|                 0|            0|                 0|       0|      0|         0|    

Nenhuma variável com atributo nulo, como previsto no último tratamento!

## b. Preparração dos dados

O modddelo de clustering a ser montado também demanda a vetorizaçõ dos dados, assim como os modelos de regressão demandavam:

In [73]:
from pyspark.ml.feature import VectorAssembler

In [74]:
data.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'valor',
 'zona_central',
 'zona_norte',
 'zona_oeste',
 'zona_sul',
 'Academia',
 'animais_permitidos',
 'Churrasqueira',
 'condominio_fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'portaria_24h',
 'portao_eletronico',
 'salao_de_festas']

Desta vez, todas as colunas serão utilizadas no modelo, exceto:
*  `id`: identificador único e 
*  `bairro`: não foi transformada em variável dummy por possuir muitas atribuições

Aqui, outra especificidade: a coluna `valor` não é removida para o modelo, pois o cluster será criado de acordo com um algoritmo não supervisionado, em que não é necessário uma variável alvo. 

In [75]:
input_col = [i for i in data.columns if i not in ['id', 'bairro']]

print(input_col)

['andar', 'area_util', 'banheiros', 'quartos', 'suites', 'vaga', 'condominio', 'iptu', 'valor', 'zona_central', 'zona_norte', 'zona_oeste', 'zona_sul', 'Academia', 'animais_permitidos', 'Churrasqueira', 'condominio_fechado', 'Elevador', 'Piscina', 'Playground', 'portaria_24h', 'portao_eletronico', 'salao_de_festas']


In [76]:
assembler = VectorAssembler(inputCols=input_col, outputCol='features')

In [77]:
data_prep = assembler.transform(data).select(['features'])

In [78]:
data_prep.show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------------------+
|features                                                                                                    |
+------------------------------------------------------------------------------------------------------------+
|[11.0,166.0,5.0,4.0,4.0,2.0,2100.0,4600.0,1750000.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]|
|(23,[0,1,2,3,4,6,7,8,12,14,21],[4.0,640.0,5.0,11.0,3.0,3060.0,20030.0,3800000.0,1.0,1.0,1.0])               |
|[1.0,50.0,1.0,2.0,0.0,1.0,363.0,97.0,192000.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0]      |
|(23,[1,2,3,4,5,6,7,8,12,17],[160.0,4.0,3.0,1.0,3.0,1530.0,7440.0,3490000.0,1.0,1.0])                        |
|(23,[1,2,3,4,5,8,11,15,18,19,22],[52.0,1.0,2.0,1.0,1.0,440000.0,1.0,1.0,1.0,1.0,1.0])                       |
+------------------------------------------------------------------------------------------------------------+
o

## c. Padronização dos dados

Com a classe `StandardScaler` do PySpark, pode-se padronizar os dados, fazendo com que adquiram média 0 e desvio padrão 1, de acordo com a fórmula *z-score*:

<center>
$x_{i} = \dfrac{x_{i} - \mu}{\sigma}$
</center>

em que a $\mu$ representa a média aritmética do conjunto e $\sigma$ seu desvio padrão.

In [79]:
from pyspark.ml.feature import StandardScaler

In [80]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(data_prep)
data_scaler = scaler_model.transform(data_prep)

In [81]:
data_scaler.show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                    |scaled_features                                                                                                                                                                                                                                                                                                                                                                                    |
+-

## d. Redução de dimensionalidade

Para gerar uma combinação linear de todas as colunas e assim diminuir a dimensão do sistema, realiza-se a redução de dimensionalidade através da classe `PCA`:

In [82]:
from pyspark.ml.feature import PCA

In [83]:
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(data_scaler)
data_pca = model_pca.transform(data_scaler)

In [84]:
data_pca.select('pca_features').show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-7.914179810286645,-5.023192740643865]  |
|[-2.938732076637182,-11.039000981915983] |
|[-5.091216714107639,-0.1418972328546243] |
|[-1.3047552598967083,-5.1610096090868565]|
|[-3.3948649730270692,-1.8954056038703295]|
+-----------------------------------------+
only showing top 5 rows



# Criação de um pipeline de Machine Learning

A implementação da classe `Pipeline` facilita muito a forma como se estabelece um conjunto de transformações de dados nos scripts PySpark: 

In [85]:
from pyspark.ml import Pipeline

Como se viu nas células anteriores, o pipeline deste projeto é constituído pelos seguintes estágios:

`vectorr_assembler` > `standard_scaler` > `pca`

In [86]:
pca_pipeline = Pipeline(stages=[assembler, scaler, pca])

In [87]:
pca_pipeline_model = pca_pipeline.fit(data)

In [88]:
data_pca = pca_pipeline_model.transform(data)

In [89]:
data_pca.show(5)

+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|         bairro|condominio|   iptu|    valor|zona_central|zona_norte|zona_oeste|zona_sul|Academia|animais_permitidos|Churrasqueira|condominio_fechado|Elevador|Piscina|Playground|portaria_24h|portao_eletronico|salao_de_festas|            features|     scaled_features|        pca_features|
+--------------------+-----+---------+---------+-------+------+----+---------------+----------+-------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------

Resultado da redução:

In [90]:
data_pca.select('pca_features').show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-7.914179810286645,-5.023192740643865]  |
|[-2.938732076637182,-11.039000981915983] |
|[-5.091216714107639,-0.1418972328546243] |
|[-1.3047552598967083,-5.1610096090868565]|
|[-3.3948649730270692,-1.8954056038703295]|
+-----------------------------------------+
only showing top 5 rows



#Clusterização dos dados com o algoritmo KMeans

Com a classe `KMeans` implementa-se o algoritmo que será responsável por agrupar os dados que possuem características semelhantes (cluster).

In [91]:
from pyspark.ml.clustering import KMeans

In [92]:
SEED = 1224

O algoritmo `KMeans` precisa das seguintes entradas:

*  `featuresCol`: coluna dos dados a serem clusterizados
*  `predictionCol`: coluna de clusters a que cada dado pertence
*  `k`: número de cluster a serem criados
*  `seed`: número para geração de números pseudoaletórios


In [93]:
kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca', k=5, seed=SEED)

In [94]:
model_kmeans = kmeans.fit(data_pca)

In [95]:
predictions_kmeans = model_kmeans.transform(data_pca)

In [96]:
predictions_kmeans.select('pca_features', 'cluster_pca')\
    .show(5, truncate=False)

+-----------------------------------------+-----------+
|pca_features                             |cluster_pca|
+-----------------------------------------+-----------+
|[-7.914179810286645,-5.023192740643865]  |1          |
|[-2.938732076637182,-11.039000981915983] |3          |
|[-5.091216714107639,-0.1418972328546243] |2          |
|[-1.3047552598967083,-5.1610096090868565]|3          |
|[-3.3948649730270692,-1.8954056038703295]|4          |
+-----------------------------------------+-----------+
only showing top 5 rows



## a. Análise dos clusters criados

A query a seguir mostra um resumo para informações dos clusters de acordo com as informações dos cômodos e de tamanho do imóvel:

In [36]:
data_pca\
    .join(predictions_kmeans.select('id', 'cluster_pca'), on='id')\
    .groupBy('cluster_pca')\
    .agg(
        f.count('id').alias('quantidade'),
        f.mean('valor').alias('valor_medio'),
        f.mean('area_util').alias('area_media'),
        f.round(f.mean('quartos'),0).alias('quartos_medio'),
        f.round(f.mean('vaga'), 0).alias('vagas_medio'),
        f.round(f.mean('banheiros'), 0).alias('banheiros_medio'),
        f.round(f.mean('suites'), 0).alias('suites_medio'),
        f.mean('condominio').alias('condominio_medio'),
        f.mean('iptu').alias('iptu_medio'),
    )\
    .orderBy('cluster_pca')\
    .show()

+-----------+----------+-----------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|cluster_pca|quantidade|      valor_medio|        area_media|quartos_medio|vagas_medio|banheiros_medio|suites_medio|  condominio_medio|        iptu_medio|
+-----------+----------+-----------------+------------------+-------------+-----------+---------------+------------+------------------+------------------+
|          0|     17606|805744.7303760082| 83.96370555492446|          2.0|        1.0|            2.0|         0.0|1672.8136998750426|1727.7102124275816|
|          1|      5164|3864096.858249419|283.64000774593336|          4.0|        3.0|            5.0|         3.0|16689.941518202944|15476.259101471727|
|          2|     20536|858257.9870471368| 90.33117452278925|          2.0|        1.0|            2.0|         1.0| 4352.482226334242|3669.3767530190885|
|          3|      8390|2683309.111203814|206.60703218116805|         

## b. Melhoria do modelo com a métrica de variância explicada 

Como atividade extra, verifica-se a quantidade ideal de componentes que os dados devem possuir após a transformação PCA. Para isto, vale-se da medida de **variância explicada**.

In [37]:
pca_pipeline_model.stages[2].explainedVariance

DenseVector([0.2655, 0.1721])

In [38]:
k = len(input_col)

In [39]:
print(k)

23


In [40]:
data_pca.select('pca_features').show(5, truncate=False)

+-----------------------------------------+
|pca_features                             |
+-----------------------------------------+
|[-7.914179810286645,-5.023192740643865]  |
|[-2.938732076637182,-11.039000981915983] |
|[-5.091216714107639,-0.1418972328546243] |
|[-1.3047552598967083,-5.1610096090868565]|
|[-3.3948649730270692,-1.8954056038703295]|
+-----------------------------------------+
only showing top 5 rows



In [41]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(data_pca.drop('pca_features'))
data_imoveis_pca = model_pca.transform(data_pca.drop('pca_features'))

In [42]:
data_imoveis_pca.select('pca_features').show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+---------------------------------------------------------------------------

In [43]:
round(sum(model_pca.explainedVariance)*100)

100

Como espera-se, as 23 colunas explicam  100% dos daods.

In [44]:
model_pca.explainedVariance[:]

array([2.65457494e-01, 1.72067301e-01, 9.12744818e-02, 5.43737219e-02,
       5.22298117e-02, 4.66203459e-02, 4.42659008e-02, 4.16021312e-02,
       3.46818214e-02, 2.72266440e-02, 2.43947461e-02, 2.01043117e-02,
       1.92103152e-02, 1.76122901e-02, 1.55070658e-02, 1.38947785e-02,
       1.20074124e-02, 1.13351577e-02, 1.01096341e-02, 9.23012235e-03,
       8.89844650e-03, 7.89606599e-03, 8.57717522e-15])

Soma cumulativa (`cumsum`) dos valores de variância para cada componente principal:

In [45]:
import numpy as np

In [46]:
lista_valores = np.cumsum(model_pca.explainedVariance[:])

In [47]:
print(lista_valores)

[0.26545749 0.43752479 0.52879928 0.583173   0.63540281 0.68202316
 0.72628906 0.76789119 0.80257301 0.82979965 0.8541944  0.87429871
 0.89350903 0.91112132 0.92662838 0.94052316 0.95253057 0.96386573
 0.97397537 0.98320549 0.99210393 1.         1.        ]


Com o intuito de se explicar até 70% dos dados, atualiza-se o valor de k:

In [48]:
k = sum(lista_valores <= 0.7)

print(f'Novo k: {k}')

Novo k: 6


E atualiza-se a redução também:

In [49]:
pca = PCA(k=k, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(data_pca.drop('pca_features'))
data_imoveis_pca_final = model_pca.transform(data_pca.drop('pca_features'))

In [50]:
data_imoveis_pca_final.select('pca_features').show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                            |
+------------------------------------------------------------------------------------------------------------------------+
|[-7.914179810286645,-5.023192740643865,-1.011036112444359,0.9748710818561146,-0.2847670025663553,-0.29618581901658153]  |
|[-2.938732076637182,-11.039000981915983,-4.038490438016014,2.187986153253318,-0.437860766162797,-0.09260027190872322]   |
|[-5.091216714107639,-0.1418972328546243,-0.14369304838874228,2.775392580635547,0.1370110834766442,1.0604617546093567]   |
|[-1.3047552598967083,-5.1610096090868565,-2.151348198589037,0.5930678901415252,-0.041801161519937874,1.1330543488639357]|
|[-3.3948649730270692,-1.8954056038703295,2.041885447153389,0.021934892842321724,0.08521905610799249,0.9684732130864733] |
+---------------

Validando a porcentagem de dados explicados pelo conjunto principal:

In [51]:
sum(model_pca.explainedVariance) *100

68.20231562211647

Novo pipeline:

In [52]:
pca_pipeline = Pipeline(stages=[assembler, scaler, pca])

In [53]:
model_pca_pipeline = pca_pipeline.fit(data)

In [54]:
projection = model_pca_pipeline.transform(data)

In [55]:
projection.select('pca_features').show(truncate=False, n=5)

+------------------------------------------------------------------------------------------------------------------------+
|pca_features                                                                                                            |
+------------------------------------------------------------------------------------------------------------------------+
|[-7.914179810286645,-5.023192740643865,-1.011036112444359,0.9748710818561146,-0.2847670025663553,-0.29618581901658153]  |
|[-2.938732076637182,-11.039000981915983,-4.038490438016014,2.187986153253318,-0.437860766162797,-0.09260027190872322]   |
|[-5.091216714107639,-0.1418972328546243,-0.14369304838874228,2.775392580635547,0.1370110834766442,1.0604617546093567]   |
|[-1.3047552598967083,-5.1610096090868565,-2.151348198589037,0.5930678901415252,-0.041801161519937874,1.1330543488639357]|
|[-3.3948649730270692,-1.8954056038703295,2.041885447153389,0.021934892842321724,0.08521905610799249,0.9684732130864733] |
+---------------

## c. Melhoria do modelo com a métrica do Silhoute Score

Cálculo do Silhoute Score

> Silhoutte Score é uma métrica usada para calculara o quão bom está a clusterização utilizada. Esse valor varia de -1 a 1.

*  1: Significa que os clusters estão bem separados uns dos outros.

*  0: Significa que os clusters são indiferentes, ou podemos dizer que a distância entre os clusters não é significativa.

*  -1: Significa que os clusters são atribuídos da maneira errada.

Reprodução: Alura

Aumentando-se a quantidade de cluster para 50, garante-se que todos sejam mais bem definidos, uma vez que a volumetria de dados é grande.

In [56]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]

evaluator = ClusteringEvaluator(predictionCol='cluster_pca', featuresCol='pca_features', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette_score = {}

for i in range(2,51):
    
    KMeans_algo = KMeans(k=i, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)
    
    KMeans_fit = KMeans_algo.fit(projection)
    
    output = KMeans_fit.transform(projection)
    
    score = evaluator.evaluate(output)
    silhouette_score[i] = score

In [57]:
silhouette_score

{2: 0.48064274154868714,
 3: 0.4860937907315559,
 4: 0.38161823258583105,
 5: 0.4428249783856506,
 6: 0.3746979200672158,
 7: 0.4263994145412974,
 8: 0.4743536007834268,
 9: 0.3661472020143287,
 10: 0.4695180160820781,
 11: 0.46194569424407,
 12: 0.4323722525329695,
 13: 0.4186546380724803,
 14: 0.4398037135494111,
 15: 0.5185124458287033,
 16: 0.49798351103543803,
 17: 0.47958504281783393,
 18: 0.5123974508196731,
 19: 0.4917913636609113,
 20: 0.5035444946846203,
 21: 0.5202501022252746,
 22: 0.5040977398571795,
 23: 0.502429863706091,
 24: 0.5201312985392625,
 25: 0.5050848054248894,
 26: 0.4924774938970464,
 27: 0.5245001606961731,
 28: 0.4700128652984269,
 29: 0.5008089746525817,
 30: 0.5217495600060424,
 31: 0.5163381820267522,
 32: 0.509093331017846,
 33: 0.5247714919048561,
 34: 0.5190114857535318,
 35: 0.5253519428890208,
 36: 0.521047104583288,
 37: 0.4760468027155057,
 38: 0.4958720825658593,
 39: 0.4981203158967082,
 40: 0.47775279627182016,
 41: 0.49826407458106253,
 42: 0.

In [97]:
k_best = [key for key, value in silhouette_score.items() if value == max(silhouette_score.values())]
k_best[0]

35

In [98]:
kmeans = KMeans(k=k_best[0], featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)
modelo_kmeans = kmeans.fit(projection)
projection_kmeans = modelo_kmeans.transform(projection)

In [99]:
projection_kmeans.select(['pca_features','cluster_pca', "id"]).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|pca_features                                                                                                             |cluster_pca|id                                  |
+-------------------------------------------------------------------------------------------------------------------------+-----------+------------------------------------+
|[-7.914179810286645,-5.023192740643865,-1.011036112444359,0.9748710818561146,-0.2847670025663553,-0.29618581901658153]   |0          |00790b85-56e3-43a5-a499-9e4c3708b95c|
|[-2.938732076637182,-11.039000981915983,-4.038490438016014,2.187986153253318,-0.437860766162797,-0.09260027190872322]    |6          |007f8099-8e1d-45f6-9cdf-dee58ea462f4|
|[-5.091216714107639,-0.1418972328546243,-0.14369304838874228,2.775392580635547,0.1370110834766442,1.0604617546093567]    |4          |

## d. Visualização de um cluster

Com o intuito de verificar a validação de um dos clusteres, escolhe-se um id aleatório e, assim, mostra-se o cluster em que ele está alocado:

In [100]:
id_imovel = '04171318-00f9-4306-b404-2a07b0a9b977'

In [101]:
projection_kmeans\
    .filter(projection_kmeans.id == id_imovel)\
    .select('cluster_pca')\
    .show()

+-----------+
|cluster_pca|
+-----------+
|         11|
+-----------+



In [102]:
cluster = projection_kmeans\
    .filter(projection_kmeans.id == id_imovel)\
    .select('cluster_pca')\
    .collect()[0][0]

In [103]:
print(cluster)

11


In [104]:
recomendacoes = projection_kmeans\
    .filter(projection_kmeans.cluster_pca == cluster)\
    .select('bairro', 'id', 'pca_features')

In [105]:
recomendacoes.show(8, truncate=False)

+------------------------+------------------------------------+--------------------------------------------------------------------------------------------------------------------------+
|bairro                  |id                                  |pca_features                                                                                                              |
+------------------------+------------------------------------+--------------------------------------------------------------------------------------------------------------------------+
|Recreio dos Bandeirantes|04171318-00f9-4306-b404-2a07b0a9b977|[-6.38607350641174,-1.7594200413402548,-0.8939008490586647,0.26974910375263605,-0.2386641202288416,-0.5796882609888316]   |
|Recreio dos Bandeirantes|0a35241d-3051-40c4-a2d9-669d32e7ac14|[-6.910302164356212,-1.178285332185398,-0.62660029468412,0.20466208470220604,-0.13689591300899812,0.044564760603646114]   |
|Recreio dos Bandeirantes|0ed4db7a-f9a8-4857-bcd5-9f09ed7ab46b|[-

#Finalização

## a. Busca pelas 10 melhores recomendações

Com o cálculo da distância euclidiana entre os elementos de um cluster, é possível elencar as 10 melhores recomendações. Para este teste, ainda se vale do `id` aleatório `id_imovel`.

In [106]:
imovel_procurado = recomendacoes\
    .filter(recomendacoes.id == id_imovel)\
    .select('pca_features')\
    .collect()[0][0]

In [107]:
print(f'Vetor: {imovel_procurado}')

Vetor: [-6.38607350641174,-1.7594200413402548,-0.8939008490586647,0.26974910375263605,-0.2386641202288416,-0.5796882609888316]


In [108]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType

In [109]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)

euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(imovel_procurado, x), FloatType())

recomendacoes\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')\
    .show(5)

+--------------------+--------------------+-----------+
|              bairro|                  id|  distancia|
+--------------------+--------------------+-----------+
|Recreio dos Bande...|04171318-00f9-430...|        0.0|
|Recreio dos Bande...|3dc65040-ff74-403...|0.015714277|
|         Jacarepaguá|9c8a005d-e127-4d8...|0.016470006|
|Recreio dos Bande...|c7dc293c-3a36-4db...|0.016532231|
|Freguesia (Jacare...|01a5aae1-6767-463...|0.020644499|
+--------------------+--------------------+-----------+
only showing top 5 rows



In [110]:
top_recomendacoes = recomendacoes\
    .withColumn('distancia', euclidean_udf('pca_features'))\
    .select('bairro', 'id', 'distancia')\
    .orderBy('distancia')

top_recomendacoes.show(10, truncate=False)

+------------------------+------------------------------------+-----------+
|bairro                  |id                                  |distancia  |
+------------------------+------------------------------------+-----------+
|Recreio dos Bandeirantes|04171318-00f9-4306-b404-2a07b0a9b977|0.0        |
|Recreio dos Bandeirantes|3dc65040-ff74-4034-af77-6b5f575ce8b6|0.015714277|
|Jacarepaguá             |9c8a005d-e127-4d8b-9c44-da0523637e5f|0.016470006|
|Recreio dos Bandeirantes|c7dc293c-3a36-4db8-8a91-3a9d37f55aaa|0.016532231|
|Freguesia (Jacarepaguá) |01a5aae1-6767-4634-9e87-490724b6ed90|0.020644499|
|Freguesia (Jacarepaguá) |f3e29161-e611-48d1-9678-82210822855a|0.023297567|
|Pechincha               |31685d3f-3b08-4027-8669-d34adb46edce|0.026400762|
|Freguesia (Jacarepaguá) |624060ee-2412-4d90-93f0-9f86c87d0898|0.03780748 |
|Pechincha               |a68a378a-a0ec-4d0f-a762-22ab663cdf85|0.038690396|
|Freguesia (Jacarepaguá) |5c253135-5609-4d63-a330-fa5b9a6fdabc|0.042061623|
+-----------

## b. Disponibilização da ref. de função recomendadora para o time de dev 

Consolidando tudo o que foi discutido e implementado, chega-se à função que o time de desenvolvimento usará como referência para o recomendador:

In [74]:
def calculate_euclidean_distance(imovel, valor):
    return euclidean(imovel, valor)


def recommender(id_imovel, dataframe_kmeans):
    cluster = dataframe_kmeans\
        .filter(dataframe_kmeans.id == id_imovel)\
        .select('cluster_pca')\
        .collect()[0][0]

    imoveis_recomendadas = dataframe_kmeans\
        .filter(dataframe_kmeans.cluster_pca == cluster)

    imovel_procurado = imoveis_recomendadas\
        .filter(imoveis_recomendadas.id == id_imovel)\
        .select('pca_features')\
        .collect()[0][0]

    euclidean_udf = f.udf(lambda x: calculate_euclidean_distance(
        imovel_procurado, x), FloatType())

    colunas_nao_utilizadas = [
        'features', 'scaled_features', 'pca_features', 'cluster_pca', 'distancia']

    recomendadas = imoveis_recomendadas\
        .withColumn('distancia', euclidean_udf('pca_features'))\
        .select([col for col in imoveis_recomendadas.columns if col not in colunas_nao_utilizadas])\
        .orderBy('distancia')

    return recomendadas

Realiza-se um teste para recomendar imóveis parecidos com o imóvel `id = '00b23c6d-0e9d-4be3-a4cc-cf64ca06f3d2'`

In [75]:
id_validacao = '00b23c6d-0e9d-4be3-a4cc-cf64ca06f3d2'

In [76]:
recommender(id_validacao, projection_kmeans)\
    .select('id', 'bairro')\
    .show(10, truncate=False)

+------------------------------------+---------------+
|id                                  |bairro         |
+------------------------------------+---------------+
|00b23c6d-0e9d-4be3-a4cc-cf64ca06f3d2|Parada de Lucas|
|03b3fee4-fd36-48c5-8911-00bfdfc79762|Tomás Coelho   |
|db0ad902-d0d0-4233-8a67-63428d698023|Todos os Santos|
|69e53fc2-2885-44d5-a675-9d09750f879a|Cachambi       |
|f32d7f78-f06f-4b43-9d98-62b3d7dc5d39|Tijuca         |
|d51fd7fc-ab7e-4755-9e91-0fb347ed1541|São Cristóvão  |
|b5ac5c83-4d2f-44f7-b1b6-97764decf782|Vila Isabel    |
|d7dfc69b-b974-4301-9cde-61ea052423c8|Engenho Novo   |
|2e0a73cb-5c7a-4cf8-8ded-58d0ccab3cba|Todos os Santos|
|5eb84915-78e1-4748-8b0a-a1feba703210|Andaraí        |
+------------------------------------+---------------+
only showing top 10 rows

